In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import layers
from tensorflow import keras

# Define a function to create your Keras MLP model with hyperparameters as arguments
def create_mlp_model(hidden_units=100, dropout_rate=0.3, optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.Dense(hidden_units, input_dim=53, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(8, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_mlp_model_with_normalization(hidden_units=100, dropout_rate=0.3, optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.Dense(300, input_dim=53, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(8, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_cnn_model(conv_filters=32, filter_size=3, pool_size=2, optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(53, 1)))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(8, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_rnn_model(recurrent_units=64, optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.SimpleRNN(64, input_shape=(53, 1)))
    model.add(layers.Dense(8, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_gru_model(optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.GRU(64, input_shape=(53, 1)))
    model.add(layers.Dense(8, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_ann_model(hidden_units=100, optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.Dense(300, input_dim=53, activation='relu'))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(8, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_ann_model_with_dropout(hidden_units=100, dropout_rate=0.3, optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.Dense(300, input_dim=53, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(8, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    return model


#Define Hyper Parameters Per-Model
# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=[
    'Model', 'Training_Time', 'Best_Parameters', 'Best_Accuracy'
])

model_param_grids = {
    "create_mlp_model": {  # Corrected model name
        'hidden_units': [100, 200, 300],
        'dropout_rate': [0.0, 0.2, 0.3],
        'optimizer': ['adam', 'sgd', 'rmsprop'],
        'epochs': [10, 20, 30]
    },
    "create_mlp_model_with_normalization": {  # Corrected model name
        'hidden_units': [100, 200, 300],
        'dropout_rate': [0.0, 0.2, 0.3],
        'optimizer': ['adam', 'sgd', 'rmsprop'],
        'epochs': [10, 20, 30]
    },
    "create_rnn_model": {  # Corrected model name
        'recurrent_units': [32, 64, 128],
        'optimizer': ['adam', 'sgd', 'rmsprop'],
        'epochs': [10, 20, 30]
    },
    "create_ann_model": {  # Corrected model name
        'hidden_units': [100, 200, 300],
        'optimizer': ['adam', 'sgd', 'rmsprop'],
        'epochs': [10, 20, 30]
    },
    "create_ann_model_with_dropout": {  # Corrected model name
        'hidden_units': [100, 200, 300],
        'dropout_rate': [0.0, 0.2, 0.3],
        'optimizer': ['adam', 'sgd', 'rmsprop'],
        'epochs': [10, 20, 30]
    },
    "create_gru_model": {  # Corrected model name
        'optimizer': ['adam', 'sgd', 'rmsprop'],
        'epochs': [10, 20, 30]
    },
    "create_cnn_model": {  # Corrected model name
        'conv_filters': [32, 64, 128],
        'filter_size': [3, 5, 7],  # Corrected parameter name
        'pool_size': [2, 3],
        'optimizer': ['adam', 'sgd', 'rmsprop'],
        'epochs': [10, 20, 30]
    }
}

# Create a dictionary to store the best models and their parameters
best_models = {}

        
training = pd.read_csv('SampleScenarioData.csv')
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(scaler.fit_transform(training.drop(['Unnamed: 0','Agriculture', 'Carbon Planting', 'Environmental Plantings',
           'Wheat Biofuels','Wheat (Food, Biofuels)','Wheat (Food, Bioenergy)', 'Woody Perennials (Biofuels)', 'Woody Perennials (Bioenergy)'], axis=1)),
                                                    training[['Agriculture', 'Carbon Planting', 'Environmental Plantings',
           'Wheat Biofuels','Wheat (Food, Biofuels)','Wheat (Food, Bioenergy)', 'Woody Perennials (Biofuels)', 'Woody Perennials (Bioenergy)']], 
                                                    train_size=0.9,test_size = 0.1, random_state = 500)

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=[
    'Model', 'Training_Time', 'Best_Parameters', 'Best_Accuracy'
])

# Iterate over the models and their parameter grids
for model_name, param_grid in model_param_grids.items():
    # Create the KerasClassifier wrapper for the model
    model = KerasClassifier(build_fn=globals()[f'{model_name.lower().replace(" ", "_")}'], verbose=0)
    
    # Use GridSearchCV to search for the best hyperparameters
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5)
    grid_result = grid.fit(X_train, y_train)
    
    # Get the best parameters and accuracy
    best_params = grid_result.best_params_
    best_accuracy = grid_result.best_score_
    
    # Store the best model and its parameters
    best_models[model_name] = {'best_params': best_params, 'best_accuracy': best_accuracy}
    
    # Evaluate the best model on the test set
    best_model = grid_result.best_estimator_
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Model: {model_name}")
    print(f"Best Hyperparameters: {best_params}")
    print(f"Best Training Accuracy: {best_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print("\n")
        # Store the results in the DataFrame
    results_df = results_df.append({
        'Model': model_name,
        'Training_Time': grid_result.refit_time_,
        'Best_Parameters': best_params,
        'Best_Accuracy': best_accuracy
    }, ignore_index=True)

# Save the results DataFrame to a CSV file
results_df.to_csv('model_performance_grid_search.csv', index=False)

c:\Anaconda3\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for CumulativeSumOptions not exposed
  .format(class_name), RuntimeWarning)
c:\Anaconda3\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for RankOptions not exposed
  .format(class_name), RuntimeWarning)
c:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1ty

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


c:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 200, in __call__
    sample_weight=sample_weight)
  File "c:\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 243, in _score
    **self._kwargs)
  File "c:\Anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "c:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 202, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "c:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 93, in _check_targets
    "an

KeyboardInterrupt: 

In [9]:
training = pd.read_csv('SampleScenarioData.csv')
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(scaler.fit_transform(training.drop(['Unnamed: 0','Agriculture', 'Carbon Planting', 'Environmental Plantings',
           'Wheat Biofuels','Wheat (Food, Biofuels)','Wheat (Food, Bioenergy)', 'Woody Perennials (Biofuels)', 'Woody Perennials (Bioenergy)'], axis=1)),
                                                    training[['Agriculture', 'Carbon Planting', 'Environmental Plantings',
           'Wheat Biofuels','Wheat (Food, Biofuels)','Wheat (Food, Bioenergy)', 'Woody Perennials (Biofuels)', 'Woody Perennials (Bioenergy)']], 
                                                    train_size=0.9,test_size = 0.1, random_state = 500)

In [11]:
training

,Unnamed: 0,Agriculture,Carbon Planting,Environmental Plantings,"Wheat (Food, Biofuels)",Area_Cost,Area_Cost5,greenhouse gas emissions (kgCO2/ha/year),Fixed_depriciation_cost,Fixed_depriciation_cost_5,...,Oil Price (AUCents/Litre),Electricity Price (AU$/MWh),Biodiversity Fund (% levy on carbon plantings),Hurdle Rate,Constraint Setting,In_Agricultural,Wheat Biofuels,"Wheat (Food, Bioenergy)",Woody Perennials (Biofuels),Woody Perennials (Bioenergy)
0,0,1,0,0,0,0.00,219.8581,43.882240,0.64,0.64,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
1,1,1,0,0,0,0.00,219.8581,44.264590,0.64,0.64,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
2,2,0,1,0,0,0.00,219.8581,63.153942,0.64,0.64,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
3,3,1,0,0,0,0.00,219.8581,45.714560,0.64,0.64,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
4,4,1,0,0,0,0.00,219.8581,44.185260,0.64,0.64,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812378,812378,0,1,0,0,0.00,216.0245,1224.815900,4.22,4.22,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
812379,812379,0,0,1,0,27.58,216.0245,1532.327900,4.22,4.22,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
812380,812380,0,1,0,0,0.00,216.0245,1384.012700,4.22,4.22,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
812381,812381,0,0,1,0,27.58,216.0245,1527.021900,4.22,4.22,...,158.068,118.0132,0.0,1,0,1.0,0,0,0,0
